In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import geopandas as gpd 
from shapely.wkt import loads 
from shapely.ops import substring

def queue_run_plot(edges_df):
    edges_gdf = gpd.GeoDataFrame(edges_df, crs='epsg:3857', geometry=edges_df['geometry'].map(loads))

    ### bg+evaucees (queue+run) geometry
    background_gdf = edges_gdf.loc[(edges_gdf['queue']>0) | (edges_gdf['run']>0)].copy()
    if background_gdf.shape[0]>0:
        background_gdf['total_geom'] = background_gdf.apply(lambda x: substring(x['geometry'], 1-(x['queue']+x['run'])*8/(x['storage'] + 8), 1, normalized=True), axis=1)
        background_gdf = background_gdf.set_geometry('total_geom')
    else: 
        background_gdf = None

    ### evacuees geometry
    evaucee_gdf = edges_gdf.loc[edges_gdf['evacuees_num']>0].copy()
    if evaucee_gdf.shape[0]>0:
        evaucee_gdf['evacuee_geom'] = evaucee_gdf.apply(lambda x: substring(x['geometry'], 1-x['evacuees_num']*8/(x['storage']+8), 1, normalized=True), axis=1)
        evaucee_gdf = evaucee_gdf.set_geometry('evacuee_geom')
    else:
        evaucee_gdf = None
    return background_gdf, evaucee_gdf

In [9]:
case = 'novato'
basemap = pd.read_csv('../traffic_inputs/fairfax/new_fairfax_links_tmp.csv')
basemap = gpd.GeoDataFrame(basemap, crs='epsg:4326', geometry=basemap['geometry'].map(loads)).to_crs(3857)
storage_dict = {getattr(link,'link_id') : getattr(link,'length') * getattr(link,'lanes') for link in basemap.itertuples()}

basemap1 = pd.read_csv('/Users/apple/Desktop/清华-伯克利深圳学院/伯克利项目/Five Towns/raw_data/study area boundaries/{}_outline.csv'.format(case))
basemap1 = gpd.GeoDataFrame(basemap1, crs='epsg:4326', geometry=basemap1['WKT'].map(loads)).to_crs(3857)

evacuation_destination_area1 = pd.DataFrame()
evacuation_destination_area1['name'] = ['evacuation_destination_area']
evacuation_destination_area1['WKT'] = ['MULTIPOLYGON (((-122.4829 37.9387, -122.4829 37.9483, -122.4749 37.9483, -122.4749 37.9387,-122.4829 37.9387)))']
evacuation_destination_area1 = gpd.GeoDataFrame(evacuation_destination_area1, crs='epsg:4326', 
                                               geometry=evacuation_destination_area1['WKT'].map(loads)).to_crs(3857)

evacuation_destination_area2 = pd.DataFrame()
evacuation_destination_area2['name'] = ['evacuation_destination_area']
evacuation_destination_area2['WKT'] = ['MULTIPOLYGON (((-122.5070 38.1140, -122.5070 38.1200, -122.5000 38.1200, -122.5000 38.1140,-122.5070 38.1140)))']
evacuation_destination_area2 = gpd.GeoDataFrame(evacuation_destination_area2, crs='epsg:4326', 
                                               geometry=evacuation_destination_area2['WKT'].map(loads)).to_crs(3857)

evacuation_destination_area3 = pd.DataFrame()
evacuation_destination_area3['name'] = ['evacuation_destination_area']
evacuation_destination_area3['WKT'] = ['MULTIPOLYGON (((-122.5839 38.1601, -122.5839 38.1720, -122.5728 38.1720, -122.5728 38.1601, -122.5839 38.1601)))']
evacuation_destination_area3 = gpd.GeoDataFrame(evacuation_destination_area3, crs='epsg:4326', 
                                               geometry=evacuation_destination_area3['WKT'].map(loads)).to_crs(3857)

evacuation_destination_area4 = pd.DataFrame()
evacuation_destination_area4['name'] = ['evacuation_destination_area']
evacuation_destination_area4['WKT'] = ['MULTIPOLYGON (((-122.48950 37.84187, -122.48950 37.84722, -122.48241 37.84722, -122.48241 37.84187, -122.48950 37.84187)))']
evacuation_destination_area4 = gpd.GeoDataFrame(evacuation_destination_area4, crs='epsg:4326', 
                                               geometry=evacuation_destination_area4['WKT'].map(loads)).to_crs(3857)

for t in range(0, 46000, 500):
    try:
        edges_df = pd.read_csv('./{}/with_bg/link_stats_add_srsn_reroute_rtt/l{}_vect_at_{}_bg_1_real_departure_time.csv'.format(case,case,t))
        edges_df = edges_df[~edges_df['geometry'].isnull()]
        edges_df['link_id'] = edges_df['link_id'].astype('int')
        edges_df['storage'] = edges_df['link_id'].map(storage_dict)
        edges_df['storage'] = np.where(edges_df['storage']<18, 18, edges_df['storage'])

        evacuees_df = pd.read_csv('./{}/with_bg/agent_stats_add_srsn_reroute_rtt/{}_vect_at_{}_bg_1_real_departure_time_agent.csv'.format(case,case,t))
        evacuees_df = evacuees_df[evacuees_df['agent_id']>=120000]
        evacuees_df = evacuees_df[~evacuees_df['current_link'].isnull()]
        evacuees_df['current_link'] = evacuees_df['current_link'].astype('str')
        evacuees_df['initial_letter_current_link'] = evacuees_df['current_link'].apply(lambda x: x[0])
        evacuees_df = evacuees_df[evacuees_df['initial_letter_current_link'] != 'v']
        evacuees_df['current_link'] = evacuees_df['current_link'].astype('float')
        evacuees_df['current_link'] = evacuees_df['current_link'].astype('int')
        evacuees_df.reset_index(drop = True, inplace = True)     
        link_queueevacuees_dict = evacuees_df.loc[evacuees_df['agent_status'].isin([1,2])].groupby('current_link').size().to_dict()
        edges_df['evacuees_num'] = edges_df['link_id'].map(link_queueevacuees_dict).fillna(0)
        
    except FileNotFoundError:
        continue
    background_gdf, evaucee_gdf = queue_run_plot(edges_df)
    fig, ax = plt.subplots(figsize=(10, 10))
    basemap.plot(ax=ax, lw=0.2, color='gray')
    basemap1.plot(ax=ax, lw=0.2, color='orange',alpha = 0.8)
    evacuation_destination_area1.plot(ax=ax, lw=0.2, color='pink',alpha = 0.5)
    evacuation_destination_area2.plot(ax=ax, lw=0.2, color='pink',alpha = 0.5)
    evacuation_destination_area3.plot(ax=ax, lw=0.2, color='pink',alpha = 0.5)
    evacuation_destination_area4.plot(ax=ax, lw=0.2, color='pink',alpha = 0.5)
    if case == 'fairfax':
        ax.set_xlim([-13650000,-13621802]) #fairfax
        ax.set_ylim([4570000,4583611]) #fairfax
        ax.text(0.58, 0.1, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #fairfax
        ax.text(0, 0.55, 'fairfax', fontsize = 10, transform=ax.transAxes) #fairfax
    elif case == 'novato':
        ax.set_xlim([-13651000,-13621802]) #novato
        ax.set_ylim([4570000,4603611]) #novato
        ax.text(0.6, 0.045, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #novato
        ax.text(0.50, 0.77, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #novato
        ax.text(0.22, 0.97, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #novato
        ax.text(0, 0.62, 'novato', fontsize = 10, transform=ax.transAxes) #novato
    elif case == 'tamalpais':
        ax.set_xlim([-13646500,-13630202]) #tamalpais
        ax.set_ylim([4557000,4573611]) #tamalpais
        ax.text(0.735, 0.03, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #tamalpais
        ax.text(0.8, 0.865, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #tamalpais
        ax.text(0.2, 0.15, 'tamalpais', fontsize = 10, transform=ax.transAxes) #tamalpais
    elif case == 'woodacre':
        ax.set_xlim([-13656000,-13623802]) #woodacre
        ax.set_ylim([4570000,4583611]) #woodacre
        ax.text(0.69, 0.1, 'evacuation destination', fontsize = 10, transform=ax.transAxes) #woodacre
        ax.text(0, 0.55, 'woodacre', fontsize = 10, transform=ax.transAxes) #woodacre        
        
        
    if background_gdf is not None:
        background_gdf.plot(ax=ax, lw=1, color='darkblue', label = 'background traffic')
    if evaucee_gdf is not None:
        evaucee_gdf.plot(ax=ax, lw=1, color='darkred', label = 'evacuation traffic')
    ax.axis('off')
    ax.text(0, 0.1, 't{}'.format(t), fontsize = 18, transform=ax.transAxes)

    ax.legend()
    plt.savefig('images/{}/edges_t{}.png'.format(case,t),dpi= 200)
    plt.close()

No handles with labels found to put in legend.
/Users/apple/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/apple/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/apple/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/apple/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  

In [33]:
case = 'fairfax'
basemap = pd.read_csv('../traffic_inputs/fairfax/new_fairfax_links_tmp.csv')
basemap = gpd.GeoDataFrame(basemap, crs='epsg:4326', geometry=basemap['geometry'].map(loads)).to_crs(3857)
storage_dict = {getattr(link,'link_id') : getattr(link,'length') * getattr(link,'lanes') for link in basemap.itertuples()}

for t in range(0, 28900, 500):#28900
    try:
        edges_df = pd.read_csv('./{}/with_bg/link_stats_add_srsn_reroute_rtt/l{}_vect_at_{}_bg_1_real_departure_time.csv'.format(case,case,t))
        edges_df = edges_df[~edges_df['geometry'].isnull()]
        edges_df['link_id'] = edges_df['link_id'].astype('int')
        edges_df['storage'] = edges_df['link_id'].map(storage_dict)
        edges_df['storage'] = np.where(edges_df['storage']<18, 18, edges_df['storage'])
        edges_df['density'] = (edges_df['run'] + edges_df['queue'])*8 / edges_df['storage']
        edges_gdf = gpd.GeoDataFrame(edges_df, crs='epsg:3857', geometry=edges_df['geometry'].map(loads))
    except FileNotFoundError:
        continue

    fig, ax = plt.subplots(figsize=(10, 10))
    basemap.plot(ax=ax, lw=0.2, color='gray')
    edges_gdf.plot(column = 'density', ax = ax, lw =1, cmap='OrRd', legend = False)
    ax.set_xlim([-13645000,-13642000]) #fairfax
    ax.set_ylim([4575000,4577511]) #fairfax
    ax.axis('off')
    ax.text(0, 0.1, 't{}'.format(t), fontsize = 18, transform=ax.transAxes)
    plt.savefig('images/{}/link_density/ramp/density_t{}.png'.format(case,t),dpi= 200)
    plt.close()

/Users/apple/anaconda3/lib/python3.6/site-packages/geopandas/plotting.py:681: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,
